In [1]:
import rebound
import reboundx
import matplotlib.pyplot as plt
import numpy as np

# Collisional Fragmentation Module

With REBOUNDx you can load different collision resolve modules. The "fragmenting_collisions" module is based on simulations done by [Leinhardt & Stewart (2012)](https://iopscience.iop.org/article/10.1088/0004-637X/745/1/79/meta).

In this module, once a collision is detected, the outcome will be determined based on the impact velocity, impact angle, and the masses of the colliding bodies. We will call the more massive body "target" and the less massive "projectile". For the descision tree of collision outcomes refer to the documentation. We will briefly discuss the different collision outcomes in the next parts in this notebook.

Each particle in the simulation, has a unique ID (`fc_id`). Everytime a collision happens, the target and projectile are removed, and the particles after the collision will have new IDs. 

When you add particles in the start of the simulation, by default they will be assigned integer IDs from 1 to N, where N is the number of your particles. You can also setup the initial particle IDs if you wish to do so. 

To use the collision module, we need to provide the following inputs:

## Inputs

1. Minimum fragment mass (`fc_min_frag_mass`): Fragment masses are derived randomly from a powerlaw distribution. A minimum allowed fragment mass needs to be chosen by the user. 

2. Output file name (`fc_particle_list_file`): If you wish to print the output file you need to set the output file name (e.g. `your_output_file_name.csv`)

### Optional inputs (you can skip this if you are using the code for the first time)

1. Initial particle IDs (`fc_id`): As mentioned above, when you add particles, they will be assigned an initial integer ID from 1 to N, where N is the total number of particles. You can change this by setting `fc_id` when you add a particle (See example 3 in the notebook). 

There are also some parameters in the collision prescription that you can change if you wish to do so. If not, the code uses the default values based on [Leinhardt & Stewart (2012)](https://iopscience.iop.org/article/10.1088/0004-637X/745/1/79/meta) and [Chambers (2013)](https://www.sciencedirect.com/science/article/pii/S0019103513000754?via%3Dihub):

2. Seperation distance scale (`fc_separation_distance_scale`): When adding new fragments, they will be placed in a circle centered at the center of mass of target and projectile. The distance between the center of mass of each fragment and the center of mass of target and projectile, is $d$ times $(R_{target} + R_{projectile})$, where $d$ is the seperation distance scale. The default value used in the code is $4$. 

3. $\rho_1$ (`fc_rho1`): In the collision prescription, to compute the catastrophic disruption criterion, we will need to compute the paremeter $R_{C1}$ which is the spherical radius of the combined projectile and target masses, if they had the density $\rho_{1}$. The default value is $1000 \ kg/m^3$, which is $1.684 \times 10^6 M_{\odot}/AU^3$. For more information refer to Section 3 in [Leinhardt & Stewart (2012)](https://iopscience.iop.org/article/10.1088/0004-637X/745/1/79/meta).

4. $C^{*}$ (`fc_cstar`): $C^{*}$ is a dimensionless material parameter introduced in [Leinhardt & Stewart (2012)](https://iopscience.iop.org/article/10.1088/0004-637X/745/1/79/meta). It is a measure of the dissipation of energy within the target, and the default value for planetary mass bodies used by [Chambers (2013)](https://www.sciencedirect.com/science/article/pii/S0019103513000754?via%3Dihub) is $1.8$. 


## Outputs
Particle list file (`your_output_file_name.csv`).

Each row in this file corresponds to a "child particle" created after the collision. Each child is created from a collision at time `t`, of a certain `type` based on the flowchart. The target and projectile will be the "parents" in the collision (we call them parent1 and parent2, and that does not imply which one is target. To identify target you need to look at their masses; target has the higher mass). Once the collision happens, parents will be removed and "child" particles will be added. Therefore, each collision has unique parent IDs and can be identifies with the pair (`parent1_id`, `parent2_id`). 

We provide an example on how to use the output file to track compositions in this tutorial (add link).

The columns of the output file are as follows:


`t`: time (in your units of choice)

`type`: collision type, refer to decision tree

`new_id`: new particle's ID

`parent1_id`: new particle's parent 1 ID

`parent2_id`: new particle's parent 2 ID

`new_mass`: new particle's mass

`parent1_mass`: parent 1 mass

`parent2_mass`: parent 2 mass

`new_radius`: new particle's radius

`parent1_radius`: parent 1 radius

`parent2_radius`: parent 2 radius

`v_imp`: impact velocity

`theta_imp`: impact angle

# Example 1: two particles colliding

Now let's start the simulation and add particles.

In [2]:
sim = rebound.Simulation()
rebx = reboundx.Extras(sim)
# Load "fragmenting_collisions" as the collision resolve module
collision_resolve = rebx.load_collision_resolve("fragmenting_collisions")
rebx.add_collision_resolve(collision_resolve)

# Set up minimum fragment mass
collision_resolve.params["fc_min_frag_mass"] = 0.001

# Set up output file name
collision_resolve.params["fc_particle_list_file"] = "family_tree_single_collision.csv"

# Add particles
# This example will result in erosion of the target.

sim.add(m=0.15, r=1.0, x=0)
sim.add(m=0.10, r=1.0, x=10.0, vx=-30.0, vy=0.001, vz=0.001)


sim.particles[0].r = 1.0
sim.particles[1].r = 1.0

# Other simulation setups
sim.dt = 0.1
sim.integrator = "mercurius"
sim.collision = "direct"

print("Before collision, N = ", sim.N)
print(f"m1 = {sim.particles[0].m:.2f} and m2 = {sim.particles[1].m:.2f}")

# Integrate
integration_time = 1
sim.integrate(integration_time)

print("After collision, N = ", sim.N)
for i in range(sim.N):
    print(f"m{i+1} = {sim.particles[i].m:.2f}")

Before collision, N =  2
m1 = 0.15 and m2 = 0.10
Non grazing, lr_mass < M_t. Erosion. (Case 4, D)
After collision, N =  5
m1 = 0.09
m2 = 0.02
m3 = 0.05
m4 = 0.05
m5 = 0.04


# Example 2 : a disk of planetesimals

Now we can add more particles and create a planetesimal disk. The planetesimals will collide with one another and the collision outcome will be determined based on the decision tree. 

In [ ]:
sim = rebound.Simulation()
rebx = reboundx.Extras(sim)
# Load "fragmenting_collisions" as the collision resolve module
collision_resolve = rebx.load_collision_resolve("fragmenting_collisions")
rebx.add_collision_resolve(collision_resolve)

#sim.units = ('yr', 'AU', 'Msun')
sim.dt = 6.0/365.0
sim.rand_seed = 1
np.random.seed(1)
sim.integrator = "mercurius"
sim.collision = "direct"
sim.G = 39.476926421373

sim.add(m = 1)

n_pl = 30 # Number of planetesimals

# planetesimal mass range
lunar_mass  = 3.8e-8    # solar masses
earth_mass  = 3e-6      # solar masses
mass_min    = 0.5 * lunar_mass
mass_max    = 0.1 * earth_mass

# Set up minimum fragment mass
collision_resolve.params["fc_min_frag_mass"] = mass_min
collision_resolve.params["fc_particle_list_file"] = "family_tree_disk.csv"

rho = 5.05e6   # units of solMass/AU^3, equal to ~3 g/cm^3

# Add 30 planetary embryos
for i in range(n_pl):
    # Orbital parameters
    a = np.random.uniform(0.1, 0.5) # semi-major axis in AU
    e = np.random.uniform(0.0, 0.01) # eccentricity
    inc = np.random.uniform(0.0, np.pi/180) # inclination in radians
    omega = np.random.uniform(0.0, 2.*np.pi) # argument of periapsis
    Omega = np.random.uniform(0.0, 2.*np.pi) # longitude of ascending node
    f = np.random.uniform(0.0, 2.*np.pi) # mean anomaly
    m = np.random.uniform(mass_min, mass_max) # particle mass in solar masses
    r = ((3*m)/(4 * np.pi * rho)) ** (1/3) * 10 # particle radius in AU, times 10 to ease collisions
    sim.add(m=m, a=a, e=e, inc=inc, omega=omega, Omega=Omega, f=f, r=r)

sim.move_to_com()

In [8]:
integration_time = 1e3

sim.integrate(integration_time)

Non grazing, M_rem to small. Merging collision detected. (Case C)
Non grazing, M_rem to small. Merging collision detected. (Case C)
Elastic bounce, (Case H).


# Example 3 : a disk of planetesimals with previously set IDs

We can re-do example 2, but assign initial particle IDs when adding them to the simualtion. When doing this, you need to set up the parameter `fc_id_max` as well, which is the maximum ID assigned to the particles. This is because when new particles are added, their assigned new IDs start from `fc_id_max` and increments +1. So if you don't set this up properly, you might get the same ID for different particles. 

In [20]:
sim = rebound.Simulation()
rebx = reboundx.Extras(sim)
# Load "fragmenting_collisions" as the collision resolve module
collision_resolve = rebx.load_collision_resolve("fragmenting_collisions")
rebx.add_collision_resolve(collision_resolve)

#sim.units = ('yr', 'AU', 'Msun')
sim.dt = 6.0/365.0
sim.rand_seed = 1
np.random.seed(1)
sim.integrator = "mercurius"
sim.collision = "direct"
sim.G = 39.476926421373

sim.add(m = 1)
sim.particles[0].params["fc_id"] = 1000


n_pl = 30 # Number of planetesimals

# planetesimal mass range
lunar_mass  = 3.8e-8    # solar masses
earth_mass  = 3e-6      # solar masses
mass_min    = 0.5 * lunar_mass
mass_max    = 0.1 * earth_mass

# Set up minimum fragment mass
collision_resolve.params["fc_min_frag_mass"] = mass_min
collision_resolve.params["fc_particle_list_file"] = "family_tree_disk_set_ids.csv"

rho = 5.05e6   # units of solMass/AU^3, equal to ~3 g/cm^3

# Add 30 planetary embryos
for i in range(n_pl):
    # Orbital parameters
    a = np.random.uniform(0.1, 0.5) # semi-major axis in AU
    e = np.random.uniform(0.0, 0.01) # eccentricity
    inc = np.random.uniform(0.0, np.pi/180) # inclination in radians
    omega = np.random.uniform(0.0, 2.*np.pi) # argument of periapsis
    Omega = np.random.uniform(0.0, 2.*np.pi) # longitude of ascending node
    f = np.random.uniform(0.0, 2.*np.pi) # mean anomaly
    m = np.random.uniform(mass_min, mass_max) # particle mass in solar masses
    r = ((3*m)/(4 * np.pi * rho)) ** (1/3) * 10 # particle radius in AU, times 10 to ease collisions
    sim.add(m=m, a=a, e=e, inc=inc, omega=omega, Omega=Omega, f=f, r=r)
    sim.particles[i+1].params["fc_id"] = 1001 + i

collision_resolve.params["fc_id_max"] = sim.particles[n_pl].params["fc_id"]

sim.move_to_com()

In [23]:
print("Initial N = ", sim.N)
print("fc_id_max = ", collision_resolve.params["fc_id_max"])

Initial N =  31
fc_id_max =  1030


In [25]:
integration_time = 1e3

sim.integrate(integration_time)

Non grazing, M_rem to small. Merging collision detected. (Case C)
Non grazing, M_rem to small. Merging collision detected. (Case C)
Elastic bounce, (Case H).


In [26]:
print("Final N = ", sim.N)
print("fc_id_max (after integration) = ", collision_resolve.params["fc_id_max"])

Final N =  29
fc_id_max (after integration) =  1037


Now if you look at `family_tree_disk_set_ids.csv`, you'll see that the new IDs start from 1031, which is 1 + the maximum initial ID.